In [ ]:
import socket
import easygopigo3 as easy
gpg = easy.EasyGoPiGo3()
import math as m
import time

HOST = '10.10.10.10'  # Standard loopback interface address (localhost)
PORT = 65432 # Port to listen on (non-privileged ports are > 1023) 

scale = 10

conversion = 180/m.pi

def get_positions(grid):
    positions = []
    size = len(grid)
    k = 1
    count = "#"
    while count != 0:
        count = 0
        for i in range(size):
            for j in range(size):
                if grid[size-j-1][i] == k:
                    positions.append([i+1,j+1])
                    print("appending position number",k)
                    grid[size-j-1][i] = 0
                    count += 1
        k += 1
    print("done getting positions")
    return positions

def print_grid(grid):
    for i in grid:
        print(i)

def get_commands(pos):
    commands = []
    n = len(pos)

    for i in range(n-1):
        x_diff, y_diff = (pos[i+1][0]-pos[i][0])*scale, (pos[i+1][1]-pos[i][1])*scale

        dist = (x_diff**2+y_diff**2)**0.5

        if y_diff > 0: angle = m.atan(x_diff/y_diff)*conversion
        elif y_diff == 0 and x_diff > 0: angle = 90
        elif y_diff == 0 and x_diff < 0: angle = -90
        elif y_diff < 0 and x_diff > 0: angle = 90 - m.atan(y_diff/x_diff)*conversion
        elif y_diff < 0 and x_diff < 0: angle = - 90 - m.atan(y_diff/x_diff)*conversion
        elif x_diff == 0 and y_diff > 0: angle = 0
        elif x_diff == 0 and y_diff < 0: angle = 180
        else: angle = 0

        commands.append([round(angle,2),round(dist,2)])
    return commands

def move(com):
    current_angle = 0
    for i in com:
        distance_to_drive = float(i[1])
        angle_to_turn = float(i[0])-current_angle

        if angle_to_turn > 180:
            angle_to_turn = angle_to_turn - 360
        elif angle_to_turn < - 180:
            angle_to_turn = 360 + angle_to_turn

        print("Turning",angle_to_turn,"degrees")
        if angle_to_turn:
            gpg.orbit(angle_to_turn)
        print("Moving forward",distance_to_drive,"cm")
        gpg.drive_cm(distance_to_drive)

        current_angle = float(i[0])

def string_to_list(string_list):
    l = string_list.split("[")
    l = [ [i.strip("],")] for i in l if i]
    for j in range(len(l)):
        l[j] = l[j][0].split(",")
        for k in range(len(l[j])):
            try:
                l[j][k] = int(l[j][k].strip("], "))
            except:
                l[j].remove(l[j][k])
    l = [l[len(l)-i-1] for i in range(len(l))]
    return l

if __name__=="__main__":
    print("GO")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        while True:
            conn, addr = s.accept() 
            with conn:
                print('Connected by', addr)
                grid_string = ""
                while True:
                    rawdata = conn.recv(1024)
                    data = repr(rawdata)
                    datarefined = data[2:len(data)-1:]
                    conn.sendall(bytes("Executing", 'utf-8'))
                    if not rawdata:
                        break
                    else:
                        grid_string += datarefined
                    grid_list = string_to_list(grid_string)

                    gpg.set_speed(255)
                    print_grid(grid_list)
                    x = get_positions(grid_list)
                    print("positions:",x)
                    c = get_commands(x)
                    print("commands:",c)
                    move(c)
                    gpg.stop()
                    print("END")

